# Human Activity Recognition using Inertial sensors and Neural Networks

**Elia Bonetto, Filippo Rigotto.**

Department of Information Engineering, University of Padova, Italy.

Human Data Analytics, a.y. 2018/2019

## Part 2 - DL models

In [0]:
!nvidia-smi

In [0]:
from IPython.display import Image, clear_output
import os
from google.colab import drive
drive.mount('/content/drive/')
clear_output()
!ls /content/drive/My\ Drive/hda-project
os.chdir("/content/drive/My Drive/hda-project")

In [0]:
import json
#import logging
from datetime import datetime
import pytz

import h5py
import numpy as np
import scipy as sp
import scipy.io

import pandas as pd
pd.set_option('display.precision',3)
pd.set_option('display.float_format', '{:0.3f}'.format)

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,6)
mpl.rcParams['axes.grid'] = True

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv1D, MaxPooling1D, Dropout
#from tensorflow.keras.layers import Conv2D, ZeroPadding2D, AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
#logging.getLogger('tensorflow').disabled = True
tf.keras.backend.set_image_data_format('channels_last')

## Data loading

Start from previously preprocessed data, altrady splitted in train and test parts.

In [0]:
map_decode = {
    0: 'running',
    1: 'walking',
    2: 'jumping',
    3: 'standing',
    4: 'sitting',
    5: 'lying',
    6: 'falling'
}
num_classes = len(map_decode)

In [0]:
with h5py.File('dataset/ARS-train-test-body-framed.h5','r') as h5f:
    X_train = h5f['X_train'][:] # IMU data w.r.t body frame
    X_test  = h5f['X_test'][:]  # activities (labels)
    Y_train = h5f['Y_train'][:]
    Y_test  = h5f['Y_test'][:]

num_data = len(X_train)
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("Y_test shape:  " + str(Y_test.shape))

# categorical structures are needed for the loss function to work properly
Y_train = to_categorical(Y_train, num_classes=num_classes, dtype=np.uint8)
Y_test  = to_categorical(Y_test,  num_classes=num_classes, dtype=np.uint8)

## Models

Here we layout the Keras models of the analyzed architectures.

In [0]:
def conv1d(input_shape, num_classes):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(shape=input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    # X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> Batch Normalization -> ReLU Block applied to X
    X = Conv1D(32, 5, name='conv0')(X_input)
    X = BatchNormalization(axis=1, name='bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling1D(2, name='max_pool0')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    #X = Dense(128, activation='relu', name='fc')(X)
    X = Dense(num_classes, activation='softmax', name='softmax')(X)

    model = Model(inputs = X_input, outputs = X, name='Conv1_Model')
    return model

def conv1d_2layers(input_shape, num_classes):
    X_input = Input(input_shape)
    
    """
    if I'm using channel_last input_shape is something like
    [12000, 128, 6, 1] where 12000 is the nr of samples
    and X_input shape is like
    [128, 6, 1]
    """

    X = Conv1D(30, 5, name='conv0')(X_input)
    # in channel_last normalization is done on axis=1
    # which is the time domain
    X = BatchNormalization(axis = 1, name='bn0')(X)
    X = Activation('relu')(X)
    
    X = Dropout(0.3)(X)
    X = MaxPooling1D(2, name='max_pool0')(X)
    
    X = Conv1D(40, 5, name='conv1')(X)
    X = BatchNormalization(axis = 1, name='bn1')(X)
    X = Activation('relu')(X)
    
    X = Dropout(0.3)(X)
    X = MaxPooling1D(2, name='max_pool1')(X)
    
    X = Flatten()(X)
    X = Dense(150, activation="relu", name='fc')(X)
    X = Dropout(0.2)(X)
    X = Dense(num_classes, activation='softmax', name='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='Conv1_2layers_Model')
    return model

## Training and evaluation

The `run_model` function takes care of bootstrap, training and evaluation processes for a given model and configuration.

In [0]:
def run_model(model, config):
    """Generic method to build a model, train and evaluate performances."""

    out_folder = os.path.join('output', datetime.now(pytz.timezone('Europe/Rome')).strftime('%Y%m%d-%H%M%S')+'_'+model.name)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    
    # print and save model summary
    print('Summary')
    model.summary()
    with open(os.path.join(out_folder, 'summary.txt'),'w') as sfile:
        model.summary(print_fn=lambda x: sfile.write(x+'\n'))

    # save config
    with open(os.path.join(out_folder, 'config.json'),'w') as cfile:
        json.dump(config, cfile, indent=2)

    # compile model
    model.compile(optimizer=config['optimizer'], loss=config['loss'], metrics=['accuracy'])

    # train model, save final state and history
    print('\nTraining')
    history = model.fit(x=X_train, y=Y_train, epochs=config['epochs'], batch_size=config['batch_size'], validation_data=(X_test,Y_test))
    model.save(os.path.join(out_folder, 'model.h5'))
    with open(os.path.join(out_folder, 'history.json'),'w') as hfile:
        hpd = pd.DataFrame(history.history)
        json.dump(json.loads(hpd.to_json()), hfile, indent=2)

        #json.dump(history.history, hfile, indent=2)
        # native json module can't handle float32 objects
        # pandas can and is used as a preprocessor to json module

    
    # evaluate model, save results
    print('\nEvaluation')
    preds = model.evaluate(x=X_test, y=Y_test)
    print ("Loss = {}".format(preds[0]))
    print ("Test Accuracy = {} = {:.2f}%".format(preds[1],preds[1]*100))
    with open(os.path.join(out_folder, 'evaluation.json'),'w') as efile:
        json.dump({'loss':float(preds[0]), 'accuracy':float(preds[1])}, efile, indent=2)

    # plot and save loss and accuracy
    print('\nLoss and accuracy plots')
    plt.figure()
    plt.plot(history.history['loss'], label='Training')
    plt.plot(history.history['val_loss'], label='Validation')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-loss')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')

    plt.figure()
    plt.plot(history.history['acc'], label='Training')
    plt.plot(history.history['val_acc'], label='Validation')
    plt.legend()
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.tight_layout()
    fname = os.path.join(out_folder, 'plot-accuracy')
    plt.savefig(fname+'.png')
    plt.savefig(fname+'.pdf', format='pdf')

## Tests

Here models are trained according to config files on the dataset split.

In [0]:
config = {
    'optimizer': 'adam',
    'loss': 'categorical_crossentropy',
    'epochs': 1,
    'batch_size': 32
}

input_shape = [X_train.shape[1], X_train.shape[2]] # valid for 1D models only
model = conv1d(input_shape, num_classes)
run_model(model, config)